In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, no_op, dsift, hog, sparse_hog
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

from menpofit.sdm import CRFitter, SN, SGN
from menpofit.visualize import visualize_fitting_result

## Load training data

In [ ]:
training_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=None):
    labeller(i, 'PTS', ibug_face_66)
    i.crop_to_landmarks_proportion_inplace(0.1, group='PTS')
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    training_images.append(i)

In [ ]:
visualize_images(training_images)

## Load test data

In [ ]:
test_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/testset/', 
                           verbose=True, max_images=None):    
    i.rescale_landmarks_to_diagonal_range(200)
    i.crop_to_landmarks_proportion_inplace(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    test_images.append(i)

In [ ]:
visualize_images(test_images)

## Train Supervised Fitter

In [ ]:
cr_algorithm_cls = SN
features = [sparse_hog, sparse_hog]
patch_shape = (17, 17)
diagonal = 100
scales = (1, .5)
iterations = 10

group = 'ibug_face_66'
verbose = True
gamma = 10000

In [ ]:
cr_fitter = CRFitter(
    cr_algorithm_cls=cr_algorithm_cls,
    features=features,
    patch_shape=patch_shape,
    diagonal=diagonal, 
    scales=scales,
    iterations=iterations)
    
cr_fitter.train(    
    training_images,
    group=group,
    verbose=verbose,
    gamma=gamma)

#### Fitting loop

In [ ]:
fitters = [cr_fitter]

fitters_results = []

for j, i in enumerate(test_images):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitters[0].noisy_shape_from_shape(gt_s, noise_std=0.04)

    for fitter in fitters:
        
        fr = fitter.fit(i, s, gt_shape=gt_s) 
        fr.downscale = 0.5
        fitters_results.append(fr)
    
        print 'Image: ', j
        print fr

## Visualize Results

In [ ]:
visualize_fitting_result(fitters_results)

In [ ]:
visualize_fitting_result(fitters_results)